In [1]:
# pip install xlrd

import sys 
import os
import pandas as pd
import re

In [2]:
project_path = os.path.abspath('..')
if project_path not in sys.path:
    sys.path.append(project_path)
sys.path

['C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner\\src',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3',
 '',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner']

In [3]:
from config.config import config

In [4]:
df = pd.read_excel(config['services_excel_file'])
df.shape

(10370, 7)

In [5]:
# удаление и переименование

df = df.drop(["Ссылка.Автор.SAMAccount name", "Контрагент"], axis=1)
df.columns = ['link', 'bustype', 'service', 'comment', 'code']
df.shape

(10370, 5)

In [6]:
# преобразование в строку, замена переноса строк на пробел, удаление повторяющихся пробелов 

for col in df.columns:
    df[col] = df[col].apply(lambda x: str(x).replace('\n', ' ').strip())
    df[col] = df[col].apply(lambda x: re.sub(r' {2,}', ' ', x).strip())

In [7]:
# замена "1 в 1"

def replace1in1(row):
    if row['comment'] == '1 в 1':
        return row['service']
    else:
        return row['comment']

df['comment'] = df.apply(replace1in1, axis=1)

In [8]:
df.describe()

,link,bustype,service,comment,code
count,10370,10370,10370,10370,10370
unique,4000,2,525,519,533
top,Счет Поставщика 0С000000000000133258 от 22.04....,Логистика,Организация автотранспортной перевозки,Счета авто. Грузятся по реестрам.,190
freq,50,8468,1776,1864,1642


In [9]:
df.shape

(10370, 5)

### Удаление транспортных позиций

In [10]:
def remove_special_characters(text):
    response = re.sub(r'[^\w\s]', ' ', text.lower())
    return re.sub(r'(\s{2,}|\n)', ' ', response)

excluded = ['счета авто грузятся по реестрам', 'услуги грузятся автоматически', 'Счета авто. ']
excluded = [remove_special_characters(x) for x in excluded]
print(excluded)

['счета авто грузятся по реестрам', 'услуги грузятся автоматически', 'счета авто ']


In [11]:
df['comment_cleaned'] = df['comment'].apply(remove_special_characters)
df = df[~df['comment_cleaned'].str.strip().isin(excluded)]
df = df.drop('comment_cleaned', axis=1)
df.shape

(6951, 5)

In [12]:
df.describe()

,link,bustype,service,comment,code
count,6951,6951,6951,6951,6951
unique,2494,2,503,517,509
top,Счет Поставщика 0С000000000000133258 от 22.04....,Логистика,Проведение экспертной оценки в целях фитосанит...,услуга русс.яз для акта,ТК-003254
freq,50,5049,197,325,197


### Удаление позиций "услуга русс. яз для акта"

In [13]:
excluded = ['услуга русс.яз для акта']
excluded = [remove_special_characters(x) for x in excluded]
print(excluded)

['услуга русс яз для акта']


In [14]:
df['comment_cleaned'] = df['comment'].apply(remove_special_characters)
df = df[~df['comment_cleaned'].str.strip().isin(excluded)]
df = df.drop('comment_cleaned', axis=1)
df.shape

(6618, 5)

### Количество различных комментариев в одной услуге

In [15]:
uniq_services = df.groupby(['service']).agg(num_comments=('comment', 'count'), uniq_comments=('comment', 'nunique')).reset_index()
uniq_services.tail()

,service,num_comments,uniq_comments
459,срочное выполнение заказа,72,2
460,термоиндикатор,26,1
461,установка и снятие генератора,4,1
462,установка и снятие дизель-генератора,30,1
463,хранение 20фут. контейнера импорт,1,1


In [16]:
uniq_services.query("uniq_comments > 1")

,service,num_comments,uniq_comments
44,Выставление контейнера на досмотр или отбор проб,8,2
47,"Выявление и идентификация грибов,нематод, вред...",16,3
50,Документальный сбор,19,2
53,Документационный сбор за коносамент,82,3
71,Идентификация вредителей,5,2
74,Информационно-консультационные услуги (чел.ч),19,2
103,Комплект деревянных поддонов,29,3
112,Лабораторная гербологическая экспертиза средни...,2,2
114,Лабораторная энтомологическая экспертиза,74,3
117,Лабораторные исследования товара,126,3


### Количество различных услуг в одном комментарии

In [17]:
uniq_comments = df.groupby(['comment']).agg(num_services=('service', 'count'), uniq_services=('service', 'nunique')).reset_index()
uniq_comments.tail()

,comment,num_services,uniq_services
510,установка и снятие генератора,4,1
511,установка и снятие дизель-генератора,30,1
512,хранение 20фут. контейнера импорт,4,3
513,хранение 40фут. контейнера импорт,2,1
514,хранение 40фут. контейнера импорт контейнера и...,6,1


In [18]:
uniq_comments.query("uniq_services > 1")

,comment,num_services,uniq_services
51,"Выявление и идентификация грибов, нематод, вре...",15,2
125,Лабораторная энтомологическая экспертиза средн...,178,2
126,Лабораторная энтомологическая экспертиза средн...,68,2
135,Локальные расходы на стороне отправления,2,2
154,Оказание комплекса услуг СВХ,11,2
174,Организация выгрузки с судна в порту прибытия,2,2
277,Перемещение контейнера для целей осмотра с исп...,7,2
286,Повышающий коэффициент: Срочность (надбавка к ...,4,2
391,Транспортно-экспедиторские услуги ТрансКонтейнер,42,6
403,Транспортно-экспедиционные услуги по организац...,16,3


### Услуги с двумя и более кодами

In [19]:
service_code_counts = df.groupby('service')['code'].nunique()

services_with_multiple_codes = service_code_counts[service_code_counts > 1]
services_with_multiple_codes

service
Доставка проб в лабораторию                                                                                                    2
Перемещение контейнера для целей досмотра/осмотра                                                                              2
Погрузо-разгрузочные работы на складах, поддерживающих температурный режим кондиционного груза в целях таможенного досмотра    2
Услуги терминала по дополнительной сортировке                                                                                  2
Хранение                                                                                                                       2
Штраф                                                                                                                          2
Name: code, dtype: int64

In [20]:
grouped = df[df['service'].isin(services_with_multiple_codes.index)].groupby(['service', 'code']).size().reset_index(name='count')
display(grouped)

idx = grouped.groupby('service')['count'].idxmin()
min_count_code = grouped.loc[idx]
display(min_count_code)

,service,code,count
0,Доставка проб в лабораторию,Т2656,5
1,Доставка проб в лабораторию,ТК-008695,152
2,Перемещение контейнера для целей досмотра/осмотра,ТК-006224,17
3,Перемещение контейнера для целей досмотра/осмотра,ТК-007431,1
4,"Погрузо-разгрузочные работы на складах, поддер...",ТК-009341,1
5,"Погрузо-разгрузочные работы на складах, поддер...",ТК-009493,1
6,Услуги терминала по дополнительной сортировке,ТК-007824,2
7,Услуги терминала по дополнительной сортировке,ТК-007848,4
8,Хранение,307,8
9,Хранение,ТК-000473,25


,service,code,count
0,Доставка проб в лабораторию,Т2656,5
3,Перемещение контейнера для целей досмотра/осмотра,ТК-007431,1
4,"Погрузо-разгрузочные работы на складах, поддер...",ТК-009341,1
6,Услуги терминала по дополнительной сортировке,ТК-007824,2
8,Хранение,307,8
11,Штраф,ТК-006548,1


In [21]:
df_ = df[~df['code'].isin(min_count_code.code)]

In [22]:
df_.describe()

,link,bustype,service,comment,code
count,6600,6600,6600,6600,6600
unique,2314,2,464,513,464
top,Счет Поставщика 0С000000000000133647 от 26.04....,Логистика,Проведение экспертной оценки в целях фитосанит...,Проведение экспертной оценки в целях фитосанит...,ТК-003254
freq,50,4704,197,191,197


### Группировка

In [23]:
# old

# grouped_comment = df.groupby(['comment']).agg(
#     service_list=('service', lambda x: list(dict.fromkeys(x))),
#     codelist = ('code', lambda x: list(dict.fromkeys(x)))
# ).reset_index()
# grouped_comment['len_service'] = grouped_comment['service_list'].apply(len)
# grouped_comment['id'] = list(range(1, grouped_comment.shape[0]+1))

In [24]:
df[['comment', 'service', 'code']].head()

,comment,service,code
38,Оформление результатов,Оформление результатов,ТК-009509
39,Дегазация подкарантинной продукции в морском к...,Дегазация подкарантинной продукции в морском к...,ТК-009507
40,1 Изучение представленных документов 2 Оформле...,Лабораторные исследования товара,ТК-008940
41,1 Анализ и первичная экспертиза подкарантинной...,Лабораторные исследования товара,ТК-008940
42,Комплекс услуг: электронное уведомление о приб...,Комплекс услуг: электронное уведомление о приб...,ТК-008270


In [25]:
def custom_agg(group):
    service = list(group['service'])
    code = list(group['code'])
    service_code_zip = list(dict.fromkeys(zip(service, code)))
    service_list, code_list = zip(*service_code_zip)

    return pd.Series({'service_list': list(service_list), 'code_list': list(code_list)})

grouped_comment = df.groupby(['comment']).apply(custom_agg).reset_index()
grouped_comment['len_service'] = grouped_comment['service_list'].apply(len)
grouped_comment['id'] = list(range(1, grouped_comment.shape[0]+1))

grouped_comment.head()

,comment,service_list,code_list,len_service,id
0,1 Анализ и первичная экспертиза подкарантинной...,[Лабораторные исследования товара],[ТК-008940],1,1
1,1 Изучение представленных документов 2 Оформле...,[Лабораторные исследования товара],[ТК-008940],1,2
2,20-футовый контейнер - перевалка груженого кон...,[20-футовый контейнер - перевалка груженого ко...,[ТК-009698],1,3
3,20-футовый контейнер - перемещение груженого к...,[20-футовый контейнер - перемещение груженого ...,[ТК-009699],1,4
4,Cбор за несвоевременное высвобождение контейне...,[Cбор за несвоевременное высвобождение контейн...,[ТК-009695],1,5


In [26]:
error = "Длина service_list всегда должна быть равна длине code_list"

assert grouped_comment[grouped_comment['service_list'].apply(len) != grouped_comment['code_list'].apply(len)].size == 0, error

In [27]:
z = grouped_comment[grouped_comment["comment"] == "Транспортно-экспедиторские услуги ТрансКонтейнер"]
display(z)
display(z['service_list'].to_list())
display(z['code_list'].to_list())
display(len(z['service_list'].to_list()) == len(z['code_list'].to_list()))

,comment,service_list,code_list,len_service,id
391,Транспортно-экспедиторские услуги ТрансКонтейнер,"[Транспортно-экспедиторские услуги (шт), Погру...","[ТК-003562, ТК-005882, ТК-004317, ТК-009584, Т...",6,392


[['Транспортно-экспедиторские услуги (шт)',
  'Погрузо-разгрузочные работы',
  'Досмотр',
  'Перетарка ручная',
  'Хранение (сут)',
  'Замена пломбы']]

[['ТК-003562', 'ТК-005882', 'ТК-004317', 'ТК-009584', 'Т00000332', 'Т1510']]

True

In [28]:
grouped_comment.shape

(515, 5)

In [29]:
grouped_comment.to_json(config['unique_comments_file'], orient='records', force_ascii=False, indent=4)